In [1]:
# 原始模型
import numpy as np
import torch
from torch import nn
from torchvision import datasets, transforms
from torch import optim
from torch.utils.data import DataLoader, Dataset, random_split, Subset
from tqdm import tqdm,trange
import torch.nn.functional as F
from attitudeAngle import Update_IMU

In [3]:
classes = 5  #分类
hidden_dim = 64 # rnn隐藏单元数
lr = 0.001 # 学习率
epoches = 20 #训练次数
batch_size = 128 # 每一个训练批次数量
input_dim= 21
device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

cuda


In [4]:
def create_data_loader():
    train_data_path = "D:\\temp\\attitudeAngle_windows"
    # train_data_path = "D:\\temp\\augment_action_windows"
    datasets = ActionDatasets(train_data_path, transform=torch.tensor, target_transform=torch.tensor, pick_path='train_angle.pkl')
    test_datasets = ActionDatasets("D:\\temp\\yan1_action_windows\\attitudeAngle", transform=torch.tensor, target_transform=torch.tensor, pick_path='test_angle.pkl')
    split_rate = 0.8  # 训练集占整个数据集的比例
    train_len = int(split_rate * len(datasets))
    valid_len = len(datasets) - train_len

    train_sets, valid_sets = random_split(datasets, [train_len, valid_len], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_sets, batch_size=batch_size, shuffle=True,drop_last=True,pin_memory=True)
    test_loader = DataLoader(test_datasets, batch_size=batch_size, shuffle=True,drop_last=True,pin_memory=True)
    valid_loader = DataLoader(valid_sets, batch_size=batch_size, shuffle=True,drop_last=True,pin_memory=True)

    print(f"训练集大小{len(train_sets)}， 验证集大小{len(valid_sets)}")
    return train_loader, test_loader, valid_loader
train_loader, test_loader, valid_loader = create_data_loader()

训练集大小63980， 验证集大小15996


In [5]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, out_dim):
        super(RNN, self).__init__()
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, 3, batch_first=True)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.3)

        self.linear1 = nn.Linear(hidden_dim, 64)
        self.linear2 = nn.Linear(64, out_dim)
        self.linear3 = nn.Linear(input_dim, hidden_dim)
        self.linear4 = nn.Linear(hidden_dim, hidden_dim)
    def forward(self, X):
        out = self.linear3(X)
        out = self.dropout2(self.linear4(out))

        out, status = self.rnn(out)
        out = F.relu(out[:,-1,:])
        out = F.relu(self.linear1(out))
        out = self.dropout1(out)
        out = self.linear2(out)

        return out

In [6]:
rnn = RNN(input_dim, hidden_dim, classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr=lr)

In [7]:
def GETACC(loader=valid_loader):
    rnn.eval()
    cnt = 0
    sum_valid_acc = 0
    sum_valid_loss = 0
    for data, label in loader:
        data = data.to(device)
        label = label.to(device)
        out = rnn(data)
        
        _, predict = torch.max(out, 1)
        
        loss = criterion(out, label)
        sum_valid_loss += loss.item()
        acc = torch.sum((predict == label).int()) / batch_size
        sum_valid_acc += acc
        cnt+=1
    
    return sum_valid_loss/cnt, sum_valid_acc/cnt

In [8]:
for epoch in range(epoches):
    i = 0
    loss_sum = 0
    bar = tqdm(train_loader)
    for ii, (data , label) in enumerate(bar):
        rnn.train()
        print(data.shape, label.shape)
        data = data.to(device)
        label = label.to(device)
        out = rnn(data)
        loss = criterion(out, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        i+=1
        loss_sum += loss.item()

        if ii == 0:
            valid_loss,valid_acc = GETACC(valid_loader)
            
            bar.set_description(f"epoch = {epoch} train_loss = {loss_sum/i} valid_loss = {valid_loss} valid_acc= {valid_acc}")


epoch = 0 train_loss = 1.6101932525634766 valid_loss = 1.6096146770061985 valid_acc= 0.19657257199287415: 100%|██████████| 499/499 [00:12<00:00, 41.30it/s]
epoch = 1 train_loss = 0.8470474481582642 valid_loss = 0.7795671460128599 valid_acc= 0.6719380021095276: 100%|██████████| 499/499 [00:10<00:00, 49.62it/s]
epoch = 2 train_loss = 0.7073093056678772 valid_loss = 0.5563744417121333 valid_acc= 0.7854082584381104: 100%|██████████| 499/499 [00:10<00:00, 48.02it/s]
epoch = 3 train_loss = 0.4376150965690613 valid_loss = 0.4670838504548996 valid_acc= 0.8266758918762207: 100%|██████████| 499/499 [00:10<00:00, 48.89it/s]
epoch = 4 train_loss = 0.3327849507331848 valid_loss = 0.4152368447713314 valid_acc= 0.8422378897666931: 100%|██████████| 499/499 [00:10<00:00, 48.70it/s]
epoch = 5 train_loss = 0.4734743535518646 valid_loss = 0.39862948932474657 valid_acc= 0.8448840379714966: 100%|██████████| 499/499 [00:09<00:00, 50.38it/s]
epoch = 6 train_loss = 0.4232863783836365 valid_loss = 0.34546633377

In [9]:
test_loss,test_acc = GETACC(test_loader)
print(f"test_loss = {test_loss}, test_acc = {test_acc}")


test_loss = 1.9610324927738734, test_acc = 0.3761160969734192
